In [1]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

plt.ion()

In [2]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)
x = x.unsqueeze(-1).pow(torch.tensor([1, 2, 3]))

In [3]:
x

tensor([[ -3.1416,   9.8696, -31.0063],
        [ -3.1384,   9.8499, -30.9133],
        [ -3.1353,   9.8301, -30.8205],
        ...,
        [  3.1353,   9.8301,  30.8205],
        [  3.1384,   9.8499,  30.9133],
        [  3.1416,   9.8696,  31.0063]])

In [4]:
y

tensor([ 8.7423e-08, -3.1430e-03, -6.2863e-03,  ...,  6.2863e-03,
         3.1430e-03, -8.7423e-08])

In [5]:
# approximate y = sin x using RL

n_states = 3
n_actions = 2

In [6]:
model = torch.nn.Sequential(
    torch.nn.Linear(n_states, 10),
    torch.nn.Linear(10, n_actions)
)

In [7]:
model(x)

tensor([[ 0.7363, -0.1028],
        [ 0.7376, -0.1008],
        [ 0.7390, -0.0989],
        ...,
        [ 3.6065,  1.8409],
        [ 3.6164,  1.8464],
        [ 3.6263,  1.8520]], grad_fn=<AddmmBackward0>)

In [8]:
class Model(nn.Module):
    def __init__(self, n_states, n_actions):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(n_states, 10)
        self.layer2 = nn.Linear(10, n_actions)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        return self.layer2(x)

In [9]:
model = Model(n_states, n_actions)

In [10]:
model(x)

tensor([[-3.0082, -3.9653],
        [-2.9995, -3.9536],
        [-2.9908, -3.9419],
        ...,
        [-5.1420, -1.1717],
        [-5.1565, -1.1721],
        [-5.1710, -1.1725]], grad_fn=<AddmmBackward0>)

In [11]:
state = torch.randn((3), dtype=torch.float)

In [12]:
state

tensor([ 1.8149,  0.3078, -0.6981])

In [13]:
model(state)

tensor([ 0.0140, -0.6868], grad_fn=<ViewBackward0>)

In [14]:
x

tensor([[ -3.1416,   9.8696, -31.0063],
        [ -3.1384,   9.8499, -30.9133],
        [ -3.1353,   9.8301, -30.8205],
        ...,
        [  3.1353,   9.8301,  30.8205],
        [  3.1384,   9.8499,  30.9133],
        [  3.1416,   9.8696,  31.0063]])

In [15]:
model(x)

tensor([[-3.0082, -3.9653],
        [-2.9995, -3.9536],
        [-2.9908, -3.9419],
        ...,
        [-5.1420, -1.1717],
        [-5.1565, -1.1721],
        [-5.1710, -1.1725]], grad_fn=<AddmmBackward0>)

In [16]:
state = torch.tensor([[-3.1, 9.8, -30.0]])

In [17]:
action = model(state).max(1).indices.view(1,1)

In [18]:
model(state)

tensor([[-2.9010, -3.8730]], grad_fn=<AddmmBackward0>)

In [19]:
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state'))

In [20]:
class Memory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
    def push(self, *args):
        self.memory.append(Transition(*args))
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    def __len__(self):
        return len(self.memory)

In [21]:
# what happens when you choose the max action? You want to go to another state. Which state is it?

In [22]:
#get step functon that returns next_state and reward
#increase or decrease y-value of y = sin x by 0.01

def step(state, action):
    if action == 1:
        next_state = state + 0.01
    else:
        next_state = state - 0.01
    reward = torch.tensor([1], dtype=torch.long)
    return next_state, reward

In [23]:
class Agent:
    def __init__(self, n_states, n_actions):
        self.batch_size = 128
        self.gamma = 0.7
        self.epsilon = 0.99
        self.decay = 0.01
        self.lr = 1e-4
        
        self.model = Model(n_states, n_actions)
        self.memory = Memory(1000)
        self.optimizer = optim.AdamW(self.model.parameters(), lr=self.lr, amsgrad=True)
    
    def get_action(self, state):
        sample = random.random()
        epsilon = self.epsilon*self.decay
        if sample > epsilon:
            with torch.no_grad():
                return self.model(state).max(1).indices.view(1, 1)
        else:
            random_action = random.choice([0, 1])
            return torch.tensor([[random_action]], dtype=torch.long)
        
    def update(self):
        if len(self.memory) < self.batch_size:
            return
        transitions = self.memory.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        next_state_batch = torch.cat(batch.next_state)
        
        q_values = self.model(state_batch).gather(1, action_batch)
        with torch.no_grad():
            next_state_values = self.model(next_state).max(1).values
        q_pred = (next_state_values * self.gamma) + reward_batch
        
        criterion = nn.SmoothL1Loss()
        loss = criterion(q_values, q_pred.unsqueeze(1))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [24]:
agent = Agent(n_states, n_actions)

episodes = []
state = torch.tensor([[-3.1, 9.8, -30.0]])

for i in range(10000):
    print(state, action)
    action = agent.get_action(state)
    next_state, reward = step(state, action)
    agent.memory.push(state, action, reward, next_state)
    state = next_state
    agent.update()

tensor([[ -3.1000,   9.8000, -30.0000]]) tensor([[0]])
tensor([[ -3.1100,   9.7900, -30.0100]]) tensor([[0]])
tensor([[ -3.1200,   9.7800, -30.0200]]) tensor([[0]])
tensor([[ -3.1300,   9.7700, -30.0300]]) tensor([[0]])
tensor([[ -3.1400,   9.7600, -30.0400]]) tensor([[0]])
tensor([[ -3.1500,   9.7500, -30.0500]]) tensor([[0]])
tensor([[ -3.1600,   9.7400, -30.0600]]) tensor([[0]])
tensor([[ -3.1700,   9.7300, -30.0700]]) tensor([[0]])
tensor([[ -3.1800,   9.7200, -30.0800]]) tensor([[0]])
tensor([[ -3.1900,   9.7100, -30.0900]]) tensor([[0]])
tensor([[ -3.2000,   9.7000, -30.1000]]) tensor([[0]])
tensor([[ -3.2100,   9.6900, -30.1100]]) tensor([[0]])
tensor([[ -3.2200,   9.6800, -30.1200]]) tensor([[0]])
tensor([[ -3.2300,   9.6700, -30.1300]]) tensor([[0]])
tensor([[ -3.2400,   9.6600, -30.1400]]) tensor([[0]])
tensor([[ -3.2500,   9.6500, -30.1500]]) tensor([[0]])
tensor([[ -3.2600,   9.6400, -30.1600]]) tensor([[0]])
tensor([[ -3.2700,   9.6300, -30.1700]]) tensor([[0]])
tensor([[ 

tensor([[ -5.9500,   6.9499, -32.8499]]) tensor([[0]])
tensor([[ -5.9600,   6.9399, -32.8599]]) tensor([[0]])
tensor([[ -5.9700,   6.9299, -32.8699]]) tensor([[0]])
tensor([[ -5.9800,   6.9199, -32.8799]]) tensor([[0]])
tensor([[ -5.9900,   6.9099, -32.8899]]) tensor([[0]])
tensor([[ -6.0000,   6.8999, -32.8999]]) tensor([[0]])
tensor([[ -6.0100,   6.8899, -32.9099]]) tensor([[0]])
tensor([[ -6.0200,   6.8799, -32.9199]]) tensor([[0]])
tensor([[ -6.0300,   6.8699, -32.9299]]) tensor([[0]])
tensor([[ -6.0400,   6.8599, -32.9399]]) tensor([[0]])
tensor([[ -6.0500,   6.8499, -32.9499]]) tensor([[0]])
tensor([[ -6.0600,   6.8399, -32.9599]]) tensor([[0]])
tensor([[ -6.0700,   6.8299, -32.9699]]) tensor([[0]])
tensor([[ -6.0800,   6.8199, -32.9799]]) tensor([[0]])
tensor([[ -6.0700,   6.8299, -32.9699]]) tensor([[1]])
tensor([[ -6.0800,   6.8199, -32.9799]]) tensor([[0]])
tensor([[ -6.0900,   6.8099, -32.9899]]) tensor([[0]])
tensor([[ -6.1000,   6.7999, -32.9999]]) tensor([[0]])
tensor([[ 

tensor([[ -7.6901,   5.2099, -34.5896]]) tensor([[0]])
tensor([[ -7.7001,   5.1999, -34.5996]]) tensor([[0]])
tensor([[ -7.7101,   5.1899, -34.6096]]) tensor([[0]])
tensor([[ -7.7201,   5.1799, -34.6196]]) tensor([[0]])
tensor([[ -7.7301,   5.1699, -34.6296]]) tensor([[0]])
tensor([[ -7.7401,   5.1599, -34.6396]]) tensor([[0]])
tensor([[ -7.7501,   5.1499, -34.6496]]) tensor([[0]])
tensor([[ -7.7601,   5.1399, -34.6596]]) tensor([[0]])
tensor([[ -7.7701,   5.1299, -34.6696]]) tensor([[0]])
tensor([[ -7.7801,   5.1199, -34.6796]]) tensor([[0]])
tensor([[ -7.7901,   5.1099, -34.6896]]) tensor([[0]])
tensor([[ -7.8001,   5.0999, -34.6996]]) tensor([[0]])
tensor([[ -7.8101,   5.0899, -34.7096]]) tensor([[0]])
tensor([[ -7.8201,   5.0799, -34.7196]]) tensor([[0]])
tensor([[ -7.8301,   5.0699, -34.7296]]) tensor([[0]])
tensor([[ -7.8401,   5.0599, -34.7396]]) tensor([[0]])
tensor([[ -7.8501,   5.0499, -34.7496]]) tensor([[0]])
tensor([[ -7.8601,   5.0399, -34.7596]]) tensor([[0]])
tensor([[ 

tensor([[ -9.6501,   3.2499, -36.5493]]) tensor([[0]])
tensor([[ -9.6601,   3.2399, -36.5593]]) tensor([[0]])
tensor([[ -9.6701,   3.2299, -36.5693]]) tensor([[0]])
tensor([[ -9.6801,   3.2199, -36.5793]]) tensor([[0]])
tensor([[ -9.6901,   3.2099, -36.5893]]) tensor([[0]])
tensor([[ -9.7001,   3.1999, -36.5993]]) tensor([[0]])
tensor([[ -9.7101,   3.1899, -36.6093]]) tensor([[0]])
tensor([[ -9.7201,   3.1799, -36.6193]]) tensor([[0]])
tensor([[ -9.7301,   3.1699, -36.6293]]) tensor([[0]])
tensor([[ -9.7401,   3.1599, -36.6393]]) tensor([[0]])
tensor([[ -9.7501,   3.1499, -36.6493]]) tensor([[0]])
tensor([[ -9.7601,   3.1399, -36.6593]]) tensor([[0]])
tensor([[ -9.7701,   3.1299, -36.6693]]) tensor([[0]])
tensor([[ -9.7801,   3.1199, -36.6793]]) tensor([[0]])
tensor([[ -9.7901,   3.1099, -36.6893]]) tensor([[0]])
tensor([[ -9.8001,   3.0999, -36.6993]]) tensor([[0]])
tensor([[ -9.8101,   3.0899, -36.7093]]) tensor([[0]])
tensor([[ -9.8201,   3.0799, -36.7193]]) tensor([[0]])
tensor([[ 

tensor([[-11.4502,   1.4499, -38.3490]]) tensor([[0]])
tensor([[-11.4602,   1.4399, -38.3590]]) tensor([[0]])
tensor([[-11.4702,   1.4299, -38.3690]]) tensor([[0]])
tensor([[-11.4802,   1.4199, -38.3790]]) tensor([[0]])
tensor([[-11.4902,   1.4099, -38.3890]]) tensor([[0]])
tensor([[-11.5002,   1.3999, -38.3990]]) tensor([[0]])
tensor([[-11.5102,   1.3899, -38.4090]]) tensor([[0]])
tensor([[-11.5202,   1.3799, -38.4190]]) tensor([[0]])
tensor([[-11.5302,   1.3699, -38.4290]]) tensor([[0]])
tensor([[-11.5402,   1.3599, -38.4390]]) tensor([[0]])
tensor([[-11.5502,   1.3499, -38.4490]]) tensor([[0]])
tensor([[-11.5602,   1.3399, -38.4590]]) tensor([[0]])
tensor([[-11.5702,   1.3299, -38.4690]]) tensor([[0]])
tensor([[-11.5802,   1.3199, -38.4790]]) tensor([[0]])
tensor([[-11.5902,   1.3099, -38.4890]]) tensor([[0]])
tensor([[-11.6002,   1.2999, -38.4990]]) tensor([[0]])
tensor([[-11.6102,   1.2899, -38.5090]]) tensor([[0]])
tensor([[-11.6202,   1.2799, -38.5190]]) tensor([[0]])
tensor([[-

...

tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-

tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-

tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-

tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7703,  -4.8701, -44.6679]]) tensor([[1]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[0]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-

tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-17.7903,  -4.8901, -44.6879]]) tensor([[0]])
tensor([[-17.7803,  -4.8801, -44.6779]]) tensor([[1]])
tensor([[-

In [25]:
state

tensor([[-17.7803,  -4.8801, -44.6779]])

In [26]:
next_state

tensor([[-17.7803,  -4.8801, -44.6779]])

In [27]:
agent.memory.sample(10)

[Transition(state=tensor([[-17.7803,  -4.8801, -44.6779]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[-17.7903,  -4.8901, -44.6879]])),
 Transition(state=tensor([[-17.7903,  -4.8901, -44.6879]]), action=tensor([[1]]), reward=tensor([1]), next_state=tensor([[-17.7803,  -4.8801, -44.6779]])),
 Transition(state=tensor([[-17.7803,  -4.8801, -44.6779]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[-17.7903,  -4.8901, -44.6879]])),
 Transition(state=tensor([[-17.7903,  -4.8901, -44.6879]]), action=tensor([[1]]), reward=tensor([1]), next_state=tensor([[-17.7803,  -4.8801, -44.6779]])),
 Transition(state=tensor([[-17.7803,  -4.8801, -44.6779]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[-17.7903,  -4.8901, -44.6879]])),
 Transition(state=tensor([[-17.7803,  -4.8801, -44.6779]]), action=tensor([[1]]), reward=tensor([1]), next_state=tensor([[-17.7703,  -4.8701, -44.6679]])),
 Transition(state=tensor([[-17.7803,  -4.8801, -44.6779]]), acti

In [28]:
agent.model.state_dict()

OrderedDict([('layer1.weight',
              tensor([[-0.0265,  0.3881, -0.1250],
                      [-0.0081,  0.1977, -0.4759],
                      [-0.4066,  0.0159, -0.0980],
                      [-0.1179,  0.1592,  0.1672],
                      [ 0.5899, -0.4302, -0.4534],
                      [-0.5045, -0.0724,  0.5330],
                      [-0.2196,  0.1465,  0.1834],
                      [ 0.4273, -0.0274,  0.4635],
                      [ 0.4013, -0.2181,  0.2311],
                      [ 0.2553, -0.5582,  0.2800]])),
             ('layer1.bias',
              tensor([-0.3614, -0.1139,  0.3942, -0.0730, -0.3649,  0.1289, -0.3166, -0.2242,
                      -0.2387,  0.4503])),
             ('layer2.weight',
              tensor([[ 0.0714,  0.2640,  0.0189, -0.2444, -0.2267,  0.1362,  0.2488,  0.3107,
                       -0.2176, -0.2034],
                      [ 0.0505,  0.0419, -0.0975, -0.2761,  0.2694, -0.3088, -0.1407,  0.2478,
                        0.0

In [29]:
# verify results
# plot results